# Lab 5: Randomized Response

***
- **FIRST name**: Muhammad Affan
- **LAST name**: Nazir
- **Student ID**: 1807593

Leave blank if individual:
- **Collaborator names**:
- **Collaborator student IDs**:
***

In today's lab, you will learn:

1. apply randomized response mechanisms;
2. obtain the estimated count of true responses;
3. apply biased randomized response mechanisms;
4. calculate the relative error.

For this lab, you'll need the dataset `adult.csv`.

### Instructions
<!-- **Deadline.**  This assignment is due at ****.  Please check the syllabus for late submissions. -->
You are expected to write clear, detailed, and complete answers when analysing your data. Lack of this may result in point deductions.

**Reminder.** You must submit your own work.  The collaboration policy for the assignments is Consultation Collaboration. You may verbally discuss concepts with your classmates, without exchanging written text, code, or detailed advice.  You must develop your own solution and submit your own work.  All sources of information used including books, websites, students you talked to, must be cited in the submission.  Please see the course FAQ document for details on this collaboration policy.  We will adhere to current Faculty of Science guidelines on dealing with suspected cases of plagiarism.

You must use this notebook to complete your assignment. You will execute the questions in the notebook. The questions might ask for a short answer in text form or for you to write and execute a piece of code. Make sure you enter your answer in either case only in the cell provided. Do not use a different cell and do not create a new cell. Creating new cells for your code is not compatible with the auto-grading system we are using and thus your assignment will not get grading properly and you will lose marks for that question.

Your submitted notebook should run on our local installation.  So if you are importing packages not listed in the notebook or using local data files not included in the assignment package, make sure the notebook is self-contained with a requirements.txt file or cells in the notebook itself to install the extra packages.  If we cannot run your notebook, you will lose 50% of the marks, and any additional marks that may be lost due to wrong answers.

### Submission Instructions
When you are done, you will submit your work from the notebook. Make sure to save your notebook before running it, and then submit on Canvas the notebook file with your work completed. Name your file with your **Student ID number** and the assignment number (ex: 1234567_L8.ipynb). Failure to do so will result in a zero!

In [43]:
# Don't change this cell; just run it.
import numpy as np
from numpy.random import default_rng
rng = default_rng()
import pandas as pd
from scipy.optimize import minimize

# These lines do some fancy plotting magic.
import matplotlib
# This is a magic function that renders the figure in the notebook, instead of displaying a dump of the figure object.
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

# Preliminaries

You should use LaTeX for mathematical expressions in your answers, if needed.

Some beginner guides for LaTeX
* http://www.docs.is.ed.ac.uk/skills/documents/3722/3722-2014.pdf
* https://www.overleaf.com/learn/latex/Learn_LaTeX_in_30_minutes

Including math expressions in a Jupyter notebook markdown cell is easy - just put the expression inside the math delimiters, dollar signs.
Example: `$f(x) = x^2 + 5$` renders as $f(x) = x^2 + 5$.  To put an expression in display mode, or on a new line, use two dollar signs on each side:  $$ f(\mathbf{x}) = \frac{x_1}{x_2}$$


## Importing and Cleaning Data

We will use a standard Machine Learning (ML) dataset used for many analyses, the "Adult Income" or simple "Adult" dataset.  This data was extracted from the 1994 US Census database.  (R. Kohavi and B. Becker. UCI adult data set.  UCI Machine Learning Repository [https://archive.ics.uci.edu/ml/datasets/adult]. Irvine, CA: University of California, School of Information and Computer Science.)  

Load the dataset `adult.csv` into a pandas dataframe.

In [44]:
# YOUR CODE HERE
df = pd.read_csv("./adult.csv")

print(df.shape)
df.head()

(32561, 15)


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [ ]:
#TEST CELL: do not delete!

Now drop the missing values

In [45]:
# YOUR CODE HERE
df.dropna()
print(df.shape)

(32561, 15)


In [ ]:
#TEST CELL: do not delete!

Now convert the values in the class column to 1 (>50K) and 0 (<=50K)

In [46]:
# YOUR CODE HERE
df["income"] = df["income"].apply(lambda x: 1 if x == '>50K' else (0 if x == "<=50K" else x))

In [ ]:
#TEST CELL: do not delete!

## Randomized Response

Now let's implement a Randomized Response mechanism.  Let's assume the query is about the income being <=50K or >50K.  The value in the `class` column is the true answer.

#### Question 1
Consider the following randomized response to the query on each row of whether the income is <=50K or >50K.  The reported answer is as follows:

Flip a fair coin.
- If it's heads, respond truthfully:  the reported answer is the same as the value in `class`.
- If it's tails, flip a fair coin again:
   - If it's heads, the reported answer has a value of 1.
   - If it's tails, the reported answer has a value of 0.
  
Implement a function for this mechanism in the code cell below.
The function should accept a value `0` or `1` and return as output the reported answer according to the randomized response above.

In [47]:
import random
def rand_resp_c1(x):
    # YOUR CODE HERE
    flip = random.randint(0,1) # 0 =Tail, 1 = Head
    if flip == 1:
      return x
    else:
      return random.randint(0,1) # Simulate second flip. return 0 if Tail, or return 1 if Head.

df['income_rrc1'] = [rand_resp_c1(x) for x in df['income']]

In [ ]:
#TEST CELL: do not delete!

### Question 2

Now get the **estimate** for the true number of people with income >50K.  Write this result into the variable `est_true_yes`.  Given the number of reported `1`'s (income >50K), we know how to estimate the proportion or number of true `1`'s.

Calculate the true number of people with income >50K (the true answer in the data we imported) and write it into the variable `true_yes`.

**Hint**: recall the derivations we did in class to analyse our randomized response:
- $n$ is the total number of participants
- $z$ is the sum of obfuscated answers
- $m$ is the sum of true answers

<p> Since we usually don't have access to $m$, how could we estimate it using $n$ and $z$? </p>

In [48]:
# YOUR CODE HERE
total_reported_one = sum(df["income_rrc1"])
est_true_yes = (2*total_reported_one) - (len(df["income_rrc1"])/2)
true_yes = sum(df["income"])

# Answer: Since we do not have access to m, the sum of true answers; we can use n and z in the formula (2*z) - (n/2) to get the estimate of m!

In [ ]:
#TEST CELL: do not delete!

#### Question 3
Now consider a more general randomized response mechanism to the query on each row of whether the income is <=50K or >50K. This more general mechanism, that takes a parameter `bias` as an argument, is as follows:


With probability `1/2 + bias`, respond truthfully.
With probability `1/2 - bias`, respond with the opposite of the true answer.

Implement a function for this mechanism.


In [49]:
import random
def rand_resp_cg(x,b):
    # YOUR CODE HERE
    if random.random() < (0.5+b):
      return x
    else:
      return abs(1-x)

In [4]:
#TEST CELL: do not delete!

Now as we did for the coin flip case we want to compare the estimated number of people with income >50K (the responses from the randomized response) with the true number (from the dataset - the `class` column).  The difference between the two values is the error in estimation.

For this generalized randomized response function, we have a parameter `b`, the bias, that is a part of the probability of responding truthfully or not - so it is a privacy parameter.  We want to analyze the error in the estimate of the number of people with income >50K, with respect to the privacy parameter, `b`.  

So let's first set up an array, that we will call `estimates`, to hold various values of `bias` that we will test, `bias` from 0.05 to 0.45.  Let's say we want to test for bias from 0.05 to 0.45 in increments of 0.05.

In [50]:
# YOUR CODE HERE
import numpy as np
estimates = np.arange(0.05, 0.50, 0.05)
print(estimates)

[0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45]


In [6]:
#TEST CELL: do not delete!

Now write code to calculate the estimated number of true `1` answers for each value of `bias`.  That is, you want to run your function above, `rand_resp_cg(x,b)` on the `class` column (`x`) and for the above values of bias (`b`) in the `estimates` array.  The 2d array `rrg` will hold the randomized response values for the `class` column for each bias value in the `estimates` array.  So, the value of `rrg[i]` will be an array of randomized responses from the `class` column that is output from `rand_resp_cg` for bias value `estimates[i]`.  So, the 2d array `rrg` should have dimensions "number of bias values you are testing" and "number of rows in the cleaned up dataset".

After the first two lines of code in the cell below, write your code to fill `rrg` with the randomized responses and `est_yes` will contain the *estimated* number of true `1` responses for each value of bias.  So `est_yes[0]` will contain the estimated number of true yes responses when the bias is 0.05.

**Hint** : this is similar to calculating `est_true_yes` in Question 2, however now you have to consider how to include the bias in your derivation.

In [51]:
rrg=np.zeros((len(estimates),len(df['income'])))
est_yes=np.zeros(len(estimates))
# YOUR CODE HERE
for i, b in enumerate(estimates):

    for j, x in enumerate(df['income']):
        rrg[i, j] = rand_resp_cg(x, b)


    S = np.sum(rrg[i])

    N = len(df['income'])

    est_yes[i] = (S - N * (0.5 - b)) / (2 * b)

In [ ]:
#TEST CELL: do not delete!

Implement a function called `rel_error` to calculate the relative error between the true number of `1`s (in the `class` column) and the estimated number (relative to the true number).  The function should accept arrays as input and output an array of relative errors.  (So it should be able to accept arrays of numbers and output an array of the relative errors.)

In [52]:
def rel_error(true,estim):
    # YOUR CODE HERE
    true = np.array(true)
    estim = np.array(estim)

    abs_error = np.abs(true - estim)
    rel_error = abs_error / true

    return rel_error

print(rel_error(np.sum(rrg,1),est_yes))

[0.45746466 0.47235767 0.40630559 0.39616818 0.35636924 0.30319006
 0.24168382 0.1768168  0.09695703]


In [ ]:
#TEST CELL: do not delete!